In [1]:
import pandas as pd
import Bio as bio

In [2]:
df = pd.read_csv("familia1.txt", sep = '\t')

In [3]:
col1 = df["accesion"]
lista = list(col1)
lista

['BCM92930.1', 'BCM90541.1', 'BCM93128.1', 'BCM93164.1']

In [4]:
df1 = pd.read_csv("celulasas.txt", sep = '\t')

In [5]:
col2 = df1["accesion"]
lista2 = list(col2)
lista2

['QEI18240.1',
 'QEI14660.1',
 'QEI11086.1',
 'ACE82688.1',
 'QEY18624.1',
 'ARU26535.1',
 'QIN33010.1',
 'QEY52202.1',
 'QIN36310.1',
 'ARM33051.1',
 'ARB93809.1',
 'CBJ12769.1',
 'AUH73568.1',
 'QII80027.1',
 'WYH50210.1',
 'WYH54262.1',
 'ACR13005.1',
 'ACR13785.1',
 'WYH44631.1']

In [6]:
from Bio import SeqIO, Entrez
import pandas as pd

# Configurar el correo electrónico y la clave API de Entrez
Entrez.email = "scminat@utn.edu.ec"
Entrez.api_key = "1e71a6fbed3171a40ea7a6cd9e0ed8298009"

# Función para obtener información de la accesión de la proteína
def get_protein_info(protein_accession):
    handle = Entrez.efetch(db="protein", id=protein_accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    # Inicializar variables para devolver
    genome_accession = None
    locus_tag = None
    ec_number = None
    protein_sequence = None
    
    for feature in record.features:
        if feature.type == "CDS":
            genome_accession = feature.qualifiers.get("coded_by", [None])[0]
            if genome_accession:
                genome_accession = genome_accession.split(":")[0]
            locus_tag = feature.qualifiers.get("locus_tag", [None])[0]
            protein_sequence = feature.qualifiers.get("translation", ["N/A"])[0]
            break
    
    return genome_accession, locus_tag, ec_number, protein_sequence

# Función principal para extraer y combinar datos de proteínas
def extract_protein_data(protein_accessions):
    protein_data = []

    for protein_accession in protein_accessions:
        try:
            genome_accession, locus_tag, ec_number, protein_sequence = get_protein_info(protein_accession)
            
            protein_entry = {
                "Protein Accession": protein_accession,
                "Genome Accession": genome_accession,
                "Locus Tag": locus_tag,
                "EC Number": ec_number,
                "Protein Sequence": protein_sequence
            }
            
            protein_data.append(protein_entry)
        except Exception as e:
            print(f"Error al procesar {protein_accession}: {e}")

    protein_df = pd.DataFrame(protein_data)
    protein_df.to_csv('protein_info.csv', index=False)
    print("Datos de proteínas guardados en 'protein_info.csv'")
    return protein_df

In [7]:
# Ejemplo de uso
protein_accessions = ['BCM92930.1', 'BCM90541.1', 'BCM93128.1', 'BCM93164.1']  # Reemplazar con tus accesiones
extract_protein_data(protein_accessions)

Datos de proteínas guardados en 'protein_info.csv'


,Protein Accession,Genome Accession,Locus Tag,EC Number,Protein Sequence
0,BCM92930.1,complement(AP024152.1,IAD21_04814,None,N/A
1,BCM90541.1,AP024152.1,IAD21_02395,None,N/A
2,BCM93128.1,complement(AP024152.1,IAD21_05016,None,N/A
3,BCM93164.1,AP024152.1,IAD21_05052,None,N/A


In [8]:
# Ejemplo de uso
protein_accessions = lista2  # Reemplazar con tus accesiones
extract_protein_data(protein_accessions)

Datos de proteínas guardados en 'protein_info.csv'


,Protein Accession,Genome Accession,Locus Tag,EC Number,Protein Sequence
0,QEI18240.1,CP043306.1,FY115_01810,None,N/A
1,QEI14660.1,CP043305.1,FY116_01810,None,N/A
2,QEI11086.1,CP043304.1,FY117_01810,None,N/A
3,ACE82688.1,CP000934.1,CJA_0374,None,N/A
4,QEY18624.1,CP031728.1,D0C16_23150,None,N/A
5,ARU26535.1,CP021382.1,CBR65_03370,None,N/A
6,QIN33010.1,CP045306.1,GCB94_13100,None,N/A
7,QEY52202.1,CP042254.1,FQU71_13760,None,N/A
8,QIN36310.1,CP045308.1,GCS73_12095,None,N/A
9,ARM33051.1,CP020894.3,B0B39_05740,None,N/A


In [9]:
from Bio import SeqIO, Entrez
import pandas as pd

# Configurar el correo electrónico y la clave API de Entrez
Entrez.email = "scminat@utn.edu.ec"
Entrez.api_key = "1e71a6fbed3171a40ea7a6cd9e0ed8298009"

# Función para obtener información del genoma completo usando la accesión y el locus
def get_genome_info(genome_accession, locus_tag):
    handle = Entrez.efetch(db="nuccore", id=genome_accession, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    
    gene_start = None
    gene_end = None
    nucleotide_sequence = None
    translated_protein_sequence = None
    
    for feature in record.features:
        if feature.type == "gene" and feature.qualifiers.get("locus_tag", [None])[0] == locus_tag:
            gene_start = int(feature.location.start)
            gene_end = int(feature.location.end)
            nucleotide_sequence = record.seq[gene_start:gene_end]
            break
    
    if not translated_protein_sequence:
        for feature in record.features:
            if feature.type == "CDS" and feature.qualifiers.get("locus_tag", [None])[0] == locus_tag:
                translated_protein_sequence = feature.qualifiers.get("translation", ["N/A"])[0]
                break
    
    return locus_tag, gene_start, gene_end, nucleotide_sequence, translated_protein_sequence

# Función principal para extraer y combinar datos del genoma
def extract_genome_data(protein_accessions):
    combined_data = []

    for protein_accession in protein_accessions:
        try:
            genome_accession, locus_tag, ec_number, protein_sequence = get_protein_info(protein_accession)
            
            if genome_accession and locus_tag:
                locus, gene_start, gene_end, nucleotide_sequence, translated_protein_sequence = get_genome_info(genome_accession, locus_tag)
                
                combined_entry = {
                    "Protein Accession": protein_accession,
                    "Genome Accession": genome_accession,
                    "Locus Tag": locus,
                    "Gene Start": gene_start,
                    "Gene End": gene_end,
                    "Nucleotide Sequence": str(nucleotide_sequence),
                    "Translated Protein Sequence": translated_protein_sequence
                }
                
                combined_data.append(combined_entry)
            else:
                print(f"No se encontró la accesión del genoma completo o el locus tag para {protein_accession}")
        except Exception as e:
            print(f"Error al procesar {protein_accession}: {e}")

    combined_df = pd.DataFrame(combined_data)
    combined_df.to_csv('genome_info.csv', index=False)
    print("Datos del genoma guardados en 'genome_info.csv'")
    return combined_df

In [10]:
# Ejemplo de uso
protein_accessions = ['BCM92930.1', 'BCM90541.1', 'BCM93128.1', 'BCM93164.1']  # Reemplazar con tus accesiones
extract_genome_data(protein_accessions)


Error al procesar BCM92930.1: HTTP Error 400: Bad Request
Error al procesar BCM93128.1: HTTP Error 400: Bad Request
Datos del genoma guardados en 'genome_info.csv'


,Protein Accession,Genome Accession,Locus Tag,Gene Start,Gene End,Nucleotide Sequence,Translated Protein Sequence
0,BCM90541.1,AP024152.1,IAD21_02395,2687530,2688790,ATGTCATCATCTATCTTCGAGTGCGAACACGCACCGCCGACAACCA...,MSSSIFECEHAPPTTNSLFKSFWMGGFECATHRRHDGRRLDCTAST...
1,BCM93164.1,AP024152.1,IAD21_05052,5785902,5787246,ATGGCCAAACAATTTCTGGACATCGTGAAAAAAAAGCTCGGCGAGA...,MAKQFLDIVKKKLGESDYSGDEHGGASGTDGSGLPDSYPGNFLFAT...


In [11]:
# Ejemplo de uso
protein_accessions = lista2  # Reemplazar con tus accesiones
extract_genome_data(protein_accessions)


Error al procesar ARB93809.1: HTTP Error 400: Bad Request
Error al procesar ACR13005.1: HTTP Error 400: Bad Request
Error al procesar WYH44631.1: HTTP Error 400: Bad Request
Datos del genoma guardados en 'genome_info.csv'


,Protein Accession,Genome Accession,Locus Tag,Gene Start,Gene End,Nucleotide Sequence,Translated Protein Sequence
0,QEI18240.1,CP043306.1,FY115_01810,472759,474295,ATGAACTTGCTATCAGGTTGGGTAAGACCTCTGATGCTGGGGTGTG...,MNLLSGWVRPLMLGCGLLGAALSAGSIQAAVCEYRVTNEWGSGFTA...
1,QEI14660.1,CP043305.1,FY116_01810,472759,474295,ATGAACTTGCTATCAGGTTGGGTAAGACCTCTGATGCTGGGGTGTG...,MNLLSGWVRPLMLGCGLLGAALSAGSIQAAVCEYRVTNEWGSGFTA...
2,QEI11086.1,CP043304.1,FY117_01810,472759,474295,ATGAACTTGCTATCAGGTTGGGTAAGACCTCTGATGCTGGGGTGTG...,MNLLSGWVRPLMLGCGLLGAALSAGSIQAAVCEYRVTNEWGSGFTA...
3,ACE82688.1,CP000934.1,CJA_0374,472759,474295,ATGAACTTGCTATCAGGTTGGGTAAGACCTCTGATGCTGGGGTGTG...,MNLLSGWVRPLMLGCGLLGAALSAGSIQAAVCEYRVTNEWGSGFTA...
4,QEY18624.1,CP031728.1,D0C16_23150,5495811,5497473,ATGAAATTGAAATCGATTTTGGAAAATGTGCCCGGCGCGTTAAGGC...,MKLKSILENVPGALRRLSLKRLSLKRPSLKRLPLKRLALGATVIIA...
5,ARU26535.1,CP021382.1,CBR65_03370,763507,765016,ATGCGCTCGGCTACGTCTTTTCTCACATCTGTCAGTCTGGGGTTAT...,MRSATSFLTSVSLGLFSWLVSSASFAQASCEYIIANEWNDGYTASI...
6,QIN33010.1,CP045306.1,GCB94_13100,2749833,2750934,ATGAAAATAGCTTCATTAAGTTTATCCGGAAGATTTAAAATAGTGT...,MKIASLSLSGRFKIVFGLLIVLTFFSQNTAYAEPENNSCAYACMDW...
7,QEY52202.1,CP042254.1,FQU71_13760,2958986,2960087,ATGAAAATAGCTTCATTAAGTTTATCCGGAAGATTTAAAATAGTGT...,MKIASLSLSGRFKIVFGLLIILTFFSQNTAYAEPENNSCAYACMDW...
8,QIN36310.1,CP045308.1,GCS73_12095,2701463,2702564,ATGAAAATAGCTTCATTAAGTTTATCCGGAAGATTTAAAATAGTGT...,MKIASLSLSGRFKIVFGLLIVLTFFSQNTAYAEPENNSCAYACMDW...
9,ARM33051.1,CP020894.3,B0B39_05740,2827831,2828932,ATGAAAATAGCTTCATTAAGTTTATCCGGAAGATTTAAAATAGTGT...,MKIASLSLSGRFKIVFGLLIVLTFFSQNTAYAEPENNSCAYACMDW...
